# Elevance Health Sweep Analysis

The inputs to the process are the following:



In [ ]:
# Input parameters
ciwpath = ('../data/SP-918447_Anlaysis.xlsx')       # CIW Download Report
rdspath = ('../data/SP-918447_Anlaysis.xlsx')       # TerraData query response

sweepPath = ('../data/SP-918447_Sweep.xlsx')         # sweep spreadsheet to process
filterDate = "15/07/2024"                           # filter date for the claims to process %d/%m/%Y
customReason = 'Claims to no longer deny for auth related denials'                      # reason before fix
payActionCodePath = ('../data/PayActionCodes.xlsx') # Pay Action Code information

Import the needed libraries

In [ ]:
import pandas as pd
import numpy as np
from re import search

Create the needed variables/constants

In [ ]:
# Sheet Names
sheetCiwReport = "CIW_Download_Report"
sheetRdsQuery = "RDS"

sheetPayAction = "PAY_ACT"
sheetStdExc = "STD Exclusions"

sheetPhase1 = "Sheet2"
sheetPhase2 = "Sheet1"
sheetPhase3 = "Sheet1"
sheetPhase4a = "MasterData"
sheetPhase4b = "Pivot"


## Import the CIW Download Report

The CIW Download report will be downloaded and saved as a sweep spreadsheet. The data from the report will be placed in Worksheet Sheet2

In [ ]:
try:
    wsCIW = pd.read_excel(ciwpath, sheet_name=sheetCiwReport)
except ValueError:
    # print("Worksheet CIW_Download_Report doesn't exist")
    raise ValueError("Worksheet CIW_Download_Report not found")
wsCIW.head(5)

Create the sweep analysis spreadsheet

In [ ]:
try:
    with pd.ExcelWriter(sweepPath,
                        mode='w') as writer:  
        wsCIW.to_excel(writer, sheet_name=sheetPhase1, header=True, index=False)
except ValueError:
    # print("Unable to create Sweep workbook")
    raise ValueError("Unable to create Sweep notebook")

## Phase 1

### LOAD _SHEET2_ OR _DATA_

In [ ]:
try:
    ws2 = pd.read_excel(sweepPath, sheet_name=sheetPhase1)
except ValueError:
    # print("Worksheet Sheet2 doesn't exist")
    try:
        ws2 = pd.read_excel(sweepPath, sheet_name='Data')
    except ValueError:
        # print("Worksheet Data doesn't exist")
        raise ValueError("Worksheet Sheet2 or Data not found")
ws2.head(5)

### CLEAR ALL FILTERS AND SELECTIONS

not needed as this was imported into a data frame

### FILTER _REMAINING_ FROM COLUMN _Status_

In [ ]:
ws2a = ws2.loc[ws2['Status'] == 'Remaining']
ws2a.head(5)

COMPUTE AND DISPLAY NO. OF ROWS WITH VALUE 'REMAINING' IN COLUMN Status

In [ ]:
print("Remaining rows: " + str(len(ws2a.index)))

COUNT TOTAL NO. OF ROWS

In [ ]:
totalRows = len(ws2[ws2['DCN'].str.len() > 0])
delRows = 0

ws2['MBU Description'] = ws2['MBU Description'].astype(str)

if 'SYSTEM_NAME' in ws2.columns:
   print("Column - SYSTEM_NAME found")

   ws2b = ws2[ws2['MBU Description'].str.contains('DEL', case=False)]
   delRows = len(ws2b.index)
   if totalRows == delRows:
      print("All DELETES, please check")

else:
   print("Column - SYSTEM_NAME not found")
    
   ws2b = ws2[ws2['MBU Description'].str.contains('DEL', case=False)]
   delRows = len(ws2b.index)
   if totalRows == delRows:
      print("All DELETES, please check")

   ws2b = ws2.loc[(ws2['Status'] == 'Remaining') & (ws2['MBU Description'].str.contains('DEL', case=False) == False)]

DISPLAY NP. OF DELETE ROWS

In [ ]:
if delRows != 0:
   print("Deletes count: " + str(delRows))

PREPARE DCN

In [ ]:
index = ws2b.columns.get_loc('DCN') + 1
ws2b.insert(index, 'Formatted DCN','')

for i, row in ws2b.iterrows():
    dcn = row['DCN']
    ws2b.at[i, 'Formatted DCN'] = "'" + dcn[2:] + "`,"

Save the changes to the spreadsheet

In [ ]:

with pd.ExcelWriter(sweepPath,
                    mode='a',
                    if_sheet_exists="replace") as writer:  
    ws2b.to_excel(writer, sheet_name=sheetPhase1, header=True, index=False)

## Run the Terradate Query

There is no connection to TerraData at this time. Import the query results

In [ ]:
try:
    wsRDS = pd.read_excel(rdspath, sheet_name=sheetRdsQuery)
except ValueError:
    # print("Worksheet RDS doesn't exist")
    raise ValueError("Worksheet RDS not found")
wsRDS.head(5)

Drop unneeded columns

In [ ]:
wsRDS.drop(wsRDS.columns[wsRDS.columns.str.contains('CASE_NBR',case = False)], axis=1, inplace=True)
wsRDS.drop(wsRDS.columns[wsRDS.columns.str.contains('SBSCRBR_ST_CD',case = False)], axis=1, inplace=True)
wsRDS.drop(wsRDS.columns[wsRDS.columns.str.contains('PROV_ST_CD',case = False)], axis=1, inplace=True)

Save the query results to the sweep workbook

In [ ]:
try:
    with pd.ExcelWriter(sweepPath,
                        mode='a',
                        if_sheet_exists="replace") as writer:  
        wsRDS.to_excel(writer, sheet_name=sheetPhase2, header=True, index=False)
except ValueError:
    # print("Unable to save RDS query results to Sweep Notebook")
    raise ValueError("Unable to save RDS query results to Sweep Notebook")

## Phase 2

LOAD SHEET1

In [ ]:
try:
    ws1 = pd.read_excel(sweepPath, sheet_name=sheetPhase2)
except ValueError:
    # print("Worksheet Sheet1 doesn't exist")
    raise ValueError("Worksheet Sheet1 not found")
ws1.head(5)

### CLEAR ALL FILTERS AND SELECTIONS

not needed as this was imported into a data frame

### READ INPUT FROM USER

not needed, this is set up as a command line parameter

### DELETE FIRST COLUM AND CREATE 3 NEW COLUMNS

In [ ]:
ws1.drop(ws1.columns[ws1.columns.str.contains('unnamed',case = False)], axis=1, inplace=True)

index = ws1.columns.get_loc('DCMNT_CNTRL_NBR') + 1
ws1.insert(index, 'Fix',' ')
ws1.insert(index, 'Reason for incl/excl',' ')
ws1.insert(index, 'IT_Analysis',' ')

###  CHANGE COLUMN FORMAT AND CREATE ANOTHER COLUMN

In [ ]:
index = ws1.columns.get_loc('PAY_ACT_CD') + 1
ws1.insert(index, 'PAY_DESC','')

### SELECT ROWS WITH VALUES IN COLUMN I STARTING IN 'F' OR 'P'

### SELECT ROWS WITH VALUES IN COLUMN J , 'EC'

### FILTER MEDICARE CLAIMS

In [ ]:
ws1.loc[ws1['ADJ_RSN'].str.contains('^F|P.*'), 'IT_Analysis'] = 'Ineligible for sweep'
ws1.loc[ws1['ADJ_RSN'].str.contains('^F|P.*'), 'Reason for incl/excl'] = 'Refunded'

ws1.loc[ws1['SRC_CD'].str.contains('^EC'), 'IT_Analysis'] = 'Ineligible for sweep'
ws1.loc[ws1['SRC_CD'].str.contains('^EC'), 'Reason for incl/excl'] = 'EC Claims'

ws1.loc[ws1['MEDCR_CNT'] == 1, 'IT_Analysis'] = 'Ineligible for sweep'
ws1.loc[ws1['MEDCR_CNT'] == 1, 'Reason for incl/excl'] = 'Medicare Claims'



### CLEAR ALL FILTERS AND SELECTIONS

### AFTER FIX

### BEFORE FIX

### PROCESSED AFTER SWEEP

### CUSTOM REASON

In [ ]:
#filterDateTime = pd.Timestamp.to_datetime(pd.to_datetime(filterDate, dayfirst=True))
ws1['Adjudicated_Date'] = pd.to_datetime(ws1['Adjudicated_Date'], dayfirst=True)

ws1.loc[ws1['Adjudicated_Date'] > filterDate, 'Fix'] = 'After fix'
ws1.loc[ws1['Adjudicated_Date'] <= filterDate, 'Fix'] = 'Before fix'

ws1.loc[(ws1['Adjudicated_Date'] > filterDate) & (ws1['IT_Analysis'].str.contains('^ $')), 'Reason for incl/excl'] = 'Processed after fix'
ws1.loc[(ws1['Adjudicated_Date'] > filterDate) & (ws1['IT_Analysis'].str.contains('^ $')), 'IT_Analysis'] = 'Ineligible for sweep'

ws1.loc[(ws1['Adjudicated_Date'] <= filterDate) & (ws1['IT_Analysis'].str.contains('^ $')), 'Reason for incl/excl'] = customReason
ws1.loc[(ws1['Adjudicated_Date'] <= filterDate) & (ws1['IT_Analysis'].str.contains('^ $')), 'IT_Analysis'] = 'Eligible for sweep'


Save changes to the spreadsheet

In [ ]:
with pd.ExcelWriter(sweepPath,
                    mode='a',
                    if_sheet_exists="replace") as writer:  
    ws1.to_excel(writer, sheet_name=sheetPhase2, header=True, index=False)

## Phase 3

LOAD SHEETS

In [ ]:
try:
    ws1 = pd.read_excel(sweepPath, 
                        sheet_name=sheetPhase3,
                        dtype={'PAY_ACT_CD': str, 'PAY_DESC': str})
except ValueError:
    # print("Worksheet Sheet1 doesn't exist")
    raise ValueError("Worksheet Sheet1 not found")

#ws1.head(5)

try:
    wsPAC = pd.read_excel(payActionCodePath, 
                          sheet_name=sheetPayAction,
                          dtype={'PAY_ACT_CD': str, 'PAY_ACT_DESC': str})
except ValueError:
    # print("Worksheet PAY_ACT doesn't exist")
    raise ValueError("Worksheet PAY_ACT not found")

#wsPAC.head(5)

try:
    wsStdEx = pd.read_excel(payActionCodePath, 
                            sheet_name=sheetStdExc,
                            dtype={'PAY_ACT_CD': str, 'PAY_ACT_DESC': str})
except ValueError:
    # print("Worksheet Std Exclusion doesn't exist")
    raise ValueError("Worksheet Std Exclusion not found")

#wsStdEx.head(5)

SELECT REQUIRED RANGE

PAY CODE TO PAY DESC CONVERSION

In [ ]:
# copy the sweep claim codes into the PAY_ACT spreadsheet
wsPAC.loc[:, 'PAY_ACT_CD'] = ws1.loc[:, 'PAY_ACT_CD']

# clear the sweep claim code descriptions
ws1.loc[:, 'PAY_DESC'] = wsPAC.loc[:, 'PAY_ACT_DESC']

STD EXCLUSION CHECK

In [ ]:
def std_exc(df1, df2):
    for id in list(df2["PAY_ACT_CD"]):
        #print('id = ' + id)
        df1.loc[df1['PAY_ACT_CD'] == id, 'IT_Analysis'] = 'Ineligible for sweep'        
        df1.loc[df1['PAY_ACT_CD'] == id, 'Reason for incl/excl'] = 'Reason for incl/excl'
    return df1

ws1 = std_exc(ws1, wsStdEx)


## Phase 4

In [ ]:
# use ws4 as a dataframe for the pivot table
# use ws1 as the master dataframe to drive the pivot table


# Save the workbook

In [ ]:
with pd.ExcelWriter(sweepPath, mode="a", if_sheet_exists="replace") as writer:   
    ws1.to_excel(writer, sheet_name=sheetPhase4a, index=False)
    #ws4.to_excel(writer, sheet_name=sheetPhase4b, index=False)